<a href="https://colab.research.google.com/github/initiative1972/data-science/blob/Natural-Language-Processing/Bert_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Importing dependencies (set up work environment)

In [1]:
import numpy as np
import math
import re #advanced text processing
import pandas as pd
from bs4 import BeautifulSoup # XML
import random

from google.colab import drive # get data from personal google drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 205 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=41e4bd494a9d1b5426cbcd9c2a5bb42eadef960ca1547f7da82caa43a8998abb
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=70d1de07b9193f3c91e3392e056cf93e9d41b4f37e7bb3caead70b870edc213c
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=f5a18d71a3b27aca857297661ae0a4489c6fb742403a81af3c1d83b109fc122b
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow p

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub # where a lot of models are hosted

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Stage 2: Data preprocessing

## Loading files

We import files from our personal Google drive.

In [5]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/projects/BERT/sentiment_data/training.csv",
    header=None,
    names=cols, # names of columns
    engine="python",
    encoding="latin1" # latin1 is common encoding; other might include UTF1
)

In [9]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [10]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Preprocessing

### Cleaning

In [11]:
def clean_tweet(tweet):
    # lxml can make use of BeautifulSoup as a parser backend, 
    # just like BeautifulSoup can employ lxml as a parser.
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links (Regex)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [13]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenization

We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [14]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [18]:
tokenizer.tokenize("My dog loves strawberries.")

['my', 'dog', 'loves', 'straw', '##berries', '.']

In [21]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [19]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries."))

[2026, 3899, 7459, 13137, 20968, 1012]

In [22]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Dataset creation

We will create padded batches (so we pad sentences for each batch inpedendently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [23]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2]) # sort by length
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]
              # remove those short sentences which do not have rich sense

In [24]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [25]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([ 2393,  6142,  2015,  4599, 21025,  2080,  2293,  8038],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [26]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [27]:
next(iter(all_batched))
# The next() function returns the next item from the iterator. If the iterator is exhausted, 
# it returns the default value passed as an argument.

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 2393,  6142,  2015,  4599, 21025,  2080,  2293,  8038],
        [ 1045,  2514, 17704,  6517,  2157,  2085,  1012,  1012],
        [ 2061,  5580,  1997,  2026,  2345,  3934,  2003,  2589],
        [ 4931,  7677, 10474,   999,  2074,  2288,  2188,  1012],
        [ 2065,  2373,  2052,  5460,  2009,  3784,  1045,  2071],
        [10094,  2000,  2377, 19456,  1997,  2162,  2006,  5282],
        [ 4067,  2017,  2005,  3582, 27439,  4710,  5254,   999],
        [ 2061,  2572,  1045,  2288,  2505,  1999,  2568,  1029],
        [ 8840,  2140,  2053,  2665, 16215, 29232,  2182,   999],
        [ 4365,  2720,  1012, 23969,  2080,  2428,  2106,  3280],
        [ 2147,  2003, 17111,  2139, 24128,  2006,  1037,  6928],
        [ 2655,  2033, 17306,  2361,  1012,  1012,  1012,  3531],
        [ 4931, 21392,  4523,   999, 10047,  1037,  2047, 11283],
        [ 2067,  2013,  1996,  2009, 11360,  1012, 26304,  2009],
        [ 5292,  3270,  2026

In [28]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Stage 3: Model building

In [29]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50, # 50 feature detector for size 2
                 # 50 feature detector for size 3
                 # 50 feature detector for size 4
                 FFN_units=512, # hidden number of units
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        # dropout: by shutting down a few neurons, prevent over-fitting
        # Thus keeping flexibility and generalization of the model
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        # call function: each time we need to make prediction using this model & function
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Stage 4: Training

In [30]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [33]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            # FFN: fit for network units
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [34]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [35]:
checkpoint_path = "./drive/MyDrive/projects/BERT/ckpt_bert_tok/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint restored!")

In [36]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [37]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
  37196/Unknown - 448s 11ms/step - loss: 0.4294 - accuracy: 0.8019Checkpoint saved at ./drive/MyDrive/projects/BERT/ckpt_bert_tok/.
37196/37196 [==============================] - 448s 11ms/step - loss: 0.4294 - accuracy: 0.8019
Epoch 2/5
37196/37196 [==============================] - 420s 11ms/step - loss: 0.3822 - accuracy: 0.8299
Epoch 3/5
37196/37196 [==============================] - 419s 11ms/step - loss: 0.3430 - accuracy: 0.8507
Epoch 4/5
37196/37196 [==============================] - 419s 11ms/step - loss: 0.3034 - accuracy: 0.8704
Epoch 5/5
37196/37196 [==============================] - 417s 11ms/step - loss: 0.2656 - accuracy: 0.8876


# Stage 5: Evaluation

In [38]:
results = Dcnn.evaluate(test_dataset)
print(results)

4132/4132 [==============================] - 27s 7ms/step - loss: 0.4641 - accuracy: 0.8278
[0.4641267955303192, 0.8278225064277649]


In [40]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False) # don't want to apply dropouts

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [41]:
get_prediction("This movie was pretty interesting.")

Output of the model: [[0.9949393]]
Predicted sentiment: positive.


In [42]:
get_prediction("I'd rather not do that again.")

Output of the model: [[0.02071244]]
Predicted sentiment: negative.


In [43]:
get_prediction("I like the dress, but not the price.")

Output of the model: [[0.14787742]]
Predicted sentiment: negative.
